# Euro-millions Number analysis
Read out the historic data to base our new numbers with and perform some data exploration.

In [1]:
import pandas as pd

df = pd.read_excel("lottery-numbers-tracker.xlsx", sheet_name="EuroMillions", header=0)

df.head()


,DrawDate,Ball 1,Ball 2,Ball 3,Ball 4,Ball 5,Lucky Star 1,Lucky Star 2
0,2023-10-13 00:00:00,21,26,28,40,41,2,4
1,2023-10-10 00:00:00,18,20,22,33,43,3,9
2,2023-10-06 00:00:00,21,29,31,34,43,2,9
3,2023-10-03 00:00:00,6,20,22,24,45,4,5
4,2023-09-29 00:00:00,9,11,13,21,32,2,7


In [2]:
# write out the data to a csv file for training a neural network
df[["Lucky Star 1", "Lucky Star 2", "Ball 1", "Ball 2", "Ball 3", "Ball 4", "Ball 5"]].to_csv("euromillions-dataset.csv", index=None)

The probability for each ball to be drawn given that there is no bias is:

In [3]:
probabilities = {i+1: 1/50 for i in range(50)}
print(probabilities)
probabilities_ls = {i+1: 1/12 for i in range(12)}
print(probabilities_ls)

{1: 0.02, 2: 0.02, 3: 0.02, 4: 0.02, 5: 0.02, 6: 0.02, 7: 0.02, 8: 0.02, 9: 0.02, 10: 0.02, 11: 0.02, 12: 0.02, 13: 0.02, 14: 0.02, 15: 0.02, 16: 0.02, 17: 0.02, 18: 0.02, 19: 0.02, 20: 0.02, 21: 0.02, 22: 0.02, 23: 0.02, 24: 0.02, 25: 0.02, 26: 0.02, 27: 0.02, 28: 0.02, 29: 0.02, 30: 0.02, 31: 0.02, 32: 0.02, 33: 0.02, 34: 0.02, 35: 0.02, 36: 0.02, 37: 0.02, 38: 0.02, 39: 0.02, 40: 0.02, 41: 0.02, 42: 0.02, 43: 0.02, 44: 0.02, 45: 0.02, 46: 0.02, 47: 0.02, 48: 0.02, 49: 0.02, 50: 0.02}
{1: 0.08333333333333333, 2: 0.08333333333333333, 3: 0.08333333333333333, 4: 0.08333333333333333, 5: 0.08333333333333333, 6: 0.08333333333333333, 7: 0.08333333333333333, 8: 0.08333333333333333, 9: 0.08333333333333333, 10: 0.08333333333333333, 11: 0.08333333333333333, 12: 0.08333333333333333}


Now we sum the number of occurrences for each ball using numpy's unique function:

In [4]:
import numpy as np
n = df.shape[0] * 5
occurrences = np.concatenate([df["Ball 1"], df["Ball 2"], df["Ball 3"], df["Ball 4"], df["Ball 5"]])

unique, counts = np.unique(occurrences, return_counts=True)
print(unique, len(unique))
running_counts = dict(zip(unique, counts))
print(running_counts)


[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50] 50
{1: 10, 2: 14, 3: 19, 4: 9, 5: 14, 6: 19, 7: 16, 8: 14, 9: 16, 10: 20, 11: 19, 12: 18, 13: 20, 14: 16, 15: 14, 16: 17, 17: 21, 18: 16, 19: 20, 20: 16, 21: 28, 22: 11, 23: 22, 24: 20, 25: 22, 26: 18, 27: 19, 28: 17, 29: 19, 30: 9, 31: 19, 32: 18, 33: 18, 34: 28, 35: 27, 36: 17, 37: 17, 38: 16, 39: 11, 40: 17, 41: 13, 42: 19, 43: 17, 44: 20, 45: 20, 46: 12, 47: 18, 48: 24, 49: 12, 50: 14}


Same unique counts for the lucky star balls:

In [5]:
n_ls = df.shape[0] * 2
occurrences_ls = np.concatenate([df["Lucky Star 1"], df["Lucky Star 2"]])

unique_ls, counts_ls = np.unique(occurrences_ls, return_counts=True)
print(unique_ls, len(unique_ls))
running_counts_ls = dict(zip(unique_ls, counts_ls))
print(running_counts_ls)


[ 1  2  3  4  5  6  7  8  9 10 11 12] 12
{1: 21, 2: 33, 3: 40, 4: 20, 5: 26, 6: 31, 7: 28, 8: 26, 9: 24, 10: 35, 11: 40, 12: 24}


Now we calculate the probability of each number and how it deviates from the raw unbiased probabilities calculated previously. It turns out the standard deviation is around a quarter of the mean, and by using the central limit theorem, we get the s.d. of the true distribution as 

In [6]:
import math
running_probabilities = {i+1: (running_counts[i + 1] / n) for i in range(50)}
print(running_probabilities)
ave = sum([running_probabilities[i + 1] for i in range(50)]) / 50.0
sd = math.sqrt(sum([(ave - running_probabilities[i + 1])**2 for i in range(50)]) / 50.0)
print(ave, sd)
print("s.d. of the true distribution", sd/math.sqrt(len(df[0:])))

{1: 0.011494252873563218, 2: 0.016091954022988506, 3: 0.021839080459770115, 4: 0.010344827586206896, 5: 0.016091954022988506, 6: 0.021839080459770115, 7: 0.01839080459770115, 8: 0.016091954022988506, 9: 0.01839080459770115, 10: 0.022988505747126436, 11: 0.021839080459770115, 12: 0.020689655172413793, 13: 0.022988505747126436, 14: 0.01839080459770115, 15: 0.016091954022988506, 16: 0.01954022988505747, 17: 0.02413793103448276, 18: 0.01839080459770115, 19: 0.022988505747126436, 20: 0.01839080459770115, 21: 0.03218390804597701, 22: 0.01264367816091954, 23: 0.02528735632183908, 24: 0.022988505747126436, 25: 0.02528735632183908, 26: 0.020689655172413793, 27: 0.021839080459770115, 28: 0.01954022988505747, 29: 0.021839080459770115, 30: 0.010344827586206896, 31: 0.021839080459770115, 32: 0.020689655172413793, 33: 0.020689655172413793, 34: 0.03218390804597701, 35: 0.03103448275862069, 36: 0.01954022988505747, 37: 0.01954022988505747, 38: 0.01839080459770115, 39: 0.01264367816091954, 40: 0.019540

This suggests there is an inherent bias in the sample of around 4 in 10000, and is consistent with why the probability of a win is a few order of magnitudes larger than it is supposed to be from a purely theoretical evaluation.
Same calculation for the lucky star balls suggests a much larger bias here:

In [7]:
running_probabilities_ls = {i+1: (running_counts_ls[i + 1] / n_ls) for i in range(12)}
print(running_probabilities_ls)
ave_ls = sum([running_probabilities_ls[i + 1] for i in range(12)]) / 12.0
sd_ls = math.sqrt(sum([(ave_ls - running_probabilities_ls[i + 1])**2 for i in range(12)]) / 12.0)
print(ave_ls, sd_ls)
print("s.d. of the true distribution", sd_ls/math.sqrt(len(df[0:])))


{1: 0.0603448275862069, 2: 0.09482758620689655, 3: 0.11494252873563218, 4: 0.05747126436781609, 5: 0.07471264367816093, 6: 0.08908045977011494, 7: 0.08045977011494253, 8: 0.07471264367816093, 9: 0.06896551724137931, 10: 0.10057471264367816, 11: 0.11494252873563218, 12: 0.06896551724137931}
0.08333333333333333 0.018770036343166114
s.d. of the true distribution 0.0014229527829659144


Now we evaluate how far apart each ball is compared to the truly unbiased mid-points of a perfectly unbiased distribution. ave_dv below is what the separation is supposed to be in a non-biased distribution but sd_dv is how that figure deviates from the ideal case, square rooted to accommodate the deviation in either direction. And it turns out the numbers tend to be less spread out from each other than the non-biased case, by around a quarter.

In [8]:
dv = 0
ddv = 0
for i in range(int(n / 5)):
    dv += sum([
        df["Ball 1"][i],
        df["Ball 2"][i] - df["Ball 1"][i],
        df["Ball 3"][i] - df["Ball 2"][i],
        df["Ball 4"][i] - df["Ball 3"][i],
        df["Ball 5"][i] - df["Ball 4"][i],
        50 - df["Ball 5"][i]
    ])
ave_dv = dv / (n * 6 / 5)

for i in range(int(n / 5)):
    ddv += sum([
        (ave_dv - df["Ball 1"][i])**2,
        (ave_dv - (df["Ball 2"][i] - df["Ball 1"][i]))**2,
        (ave_dv - (df["Ball 3"][i] - df["Ball 2"][i]))**2,
        (ave_dv - (df["Ball 4"][i] - df["Ball 3"][i]))**2,
        (ave_dv - (df["Ball 5"][i] - df["Ball 4"][i]))**2,
        (ave_dv - (50 - df["Ball 5"][i]))**2
    ])
sd_dv = math.sqrt(ddv / (n * 6 / 5))
print(ave_dv, sd_dv)


8.333333333333334 6.405517257879819


Repeat the same for lucky star balls, and it turns out there is even more bias, around a third: 

In [9]:
dv_ls = 0
ddv_ls = 0
for i in range(int(n_ls / 2)):
    dv_ls += sum([
        df["Lucky Star 1"][i],
        df["Lucky Star 2"][i] - df["Lucky Star 1"][i],
        12 - df["Lucky Star 2"][i]
    ])
ave_dv_ls = dv_ls / (n_ls * 3 / 2)

for i in range(int(n_ls / 2)):
    ddv_ls += sum([
        (ave_dv_ls - df["Lucky Star 1"][i])**2,
        (ave_dv_ls - (df["Lucky Star 2"][i] - df["Lucky Star 1"][i]))**2,
        (ave_dv_ls - (12 - df["Lucky Star 2"][i]))**2
    ])
sd_dv_ls = math.sqrt(ddv_ls / (n_ls * 3 / 2))
print(ave_dv_ls, sd_dv_ls)

4.0 2.7058897021977395


One way to make sense of this is to consider the case for lucky star balls. In a situation where there is no bias and there are infinite samples to choose from, the average probability of getting the two balls will be 4 and 8, 4 representing the smaller ball values and 8 the bigger ones. In this case the average outcome of the balls are perfectly and evenly skewed, and the distance between each draw and its neighbour is 4. that is to say 0-4, 4-8, 8-12 all have neighbouring distances of 4. 

Now consider our case where this figure is closer to 2.71, this means it is more likely to get a neighbouring average figure of 2.71, so it is more likely to get numbers like 0-2, 2-4, 4-12, which would give `sqrt((4-2)**2 + (4-2)**2 + (4-8)**2 / 3) ~ 2.82`

Now we look at repetitions in the past 10 draws for each ball:

In [10]:
rep_counts = {i+1: 0 for i in range(50)}
total_reps = 0
for i in range(int(n / 5) - 10):
    for j in range(10):
        for k in range(5):
            if (any([
                df["Ball {}".format(k + 1)][i] == df["Ball 1"][i + j + 1],
                df["Ball {}".format(k + 1)][i] == df["Ball 2"][i + j + 1],
                df["Ball {}".format(k + 1)][i] == df["Ball 3"][i + j + 1],
                df["Ball {}".format(k + 1)][i] == df["Ball 4"][i + j + 1],
                df["Ball {}".format(k + 1)][i] == df["Ball 5"][i + j + 1]
            ])):
                rep_counts[df["Ball {}".format(k + 1)][i]] += 1
                total_reps += 1
print(rep_counts)
rep_averages = {}
for i in range(50):
    rep_averages[i + 1] = rep_counts[i + 1] / total_reps
print(rep_averages)
rep_ave = sum([rep_averages[i + 1] for i in range(50)]) / 50.0
rep_sd = math.sqrt(sum([(rep_ave - rep_averages[i + 1])**2 for i in range(50)]) / 50.0)
print(rep_ave, rep_sd)

{1: 7, 2: 8, 3: 14, 4: 4, 5: 10, 6: 12, 7: 12, 8: 13, 9: 10, 10: 27, 11: 23, 12: 12, 13: 16, 14: 11, 15: 7, 16: 22, 17: 24, 18: 17, 19: 22, 20: 15, 21: 49, 22: 6, 23: 24, 24: 23, 25: 20, 26: 14, 27: 19, 28: 22, 29: 13, 30: 4, 31: 16, 32: 16, 33: 15, 34: 37, 35: 51, 36: 14, 37: 15, 38: 8, 39: 6, 40: 16, 41: 13, 42: 18, 43: 16, 44: 22, 45: 17, 46: 9, 47: 16, 48: 36, 49: 6, 50: 11}
{1: 0.008353221957040573, 2: 0.00954653937947494, 3: 0.016706443914081145, 4: 0.00477326968973747, 5: 0.011933174224343675, 6: 0.014319809069212411, 7: 0.014319809069212411, 8: 0.015513126491646777, 9: 0.011933174224343675, 10: 0.032219570405727926, 11: 0.027446300715990454, 12: 0.014319809069212411, 13: 0.01909307875894988, 14: 0.013126491646778043, 15: 0.008353221957040573, 16: 0.026252983293556086, 17: 0.028639618138424822, 18: 0.02028639618138425, 19: 0.026252983293556086, 20: 0.017899761336515514, 21: 0.05847255369928401, 22: 0.007159904534606206, 23: 0.028639618138424822, 24: 0.027446300715990454, 25: 0.0

Now the probability of a repetition for each ball is 0.2 * 1/10 attempts so roughly 0.02 which is very close to the average evaluated. However, the standard deviation is very large here, showing a tendency of numbers to repeat themselves in these smaller cycles. 

Repeating the same operation for lucky star numbers:

In [11]:
rep_counts_ls = {i+1: 0 for i in range(12)}
total_reps_ls = 0
for i in range(int(n / 5) - 10):
    for j in range(10):
        for k in range(2):
            if (any([
                df["Lucky Star {}".format(k + 1)][i] == df["Lucky Star 1"][i + j + 1],
                df["Lucky Star {}".format(k + 1)][i] == df["Lucky Star 2"][i + j + 1],
            ])):
                rep_counts_ls[df["Lucky Star {}".format(k + 1)][i]] += 1
                total_reps_ls += 1
print(rep_counts_ls)
rep_averages_ls = {}
for i in range(12):
    rep_averages_ls[i + 1] = rep_counts_ls[i + 1] / total_reps_ls
print(rep_averages_ls)
rep_ave_ls = sum([rep_averages_ls[i + 1] for i in range(12)]) / 12.0
rep_sd_ls = math.sqrt(sum([(rep_ave_ls - rep_averages_ls[i + 1])**2 for i in range(12)]) / 12.0)
print(rep_ave_ls, rep_sd_ls)


{1: 36, 2: 50, 3: 100, 4: 17, 5: 37, 6: 54, 7: 38, 8: 29, 9: 24, 10: 67, 11: 83, 12: 23}
{1: 0.06451612903225806, 2: 0.08960573476702509, 3: 0.17921146953405018, 4: 0.03046594982078853, 5: 0.06630824372759857, 6: 0.0967741935483871, 7: 0.06810035842293907, 8: 0.05197132616487455, 9: 0.043010752688172046, 10: 0.12007168458781362, 11: 0.14874551971326164, 12: 0.04121863799283154}
0.08333333333333333 0.04387022102112418


We see that the s.d. here is also quite large it is generally apparent that balls repeat themselves in cycles of 10. The degree to which this is true is still unknown but is possible to evaluate it if a running average of this figure is kept and its variance evaluated over a longer period.

Now we put together our findings and try to assign a weight to each of the evaluated probabilities in a way that the new guesses will get scores based on them. 

It is difficult to guess how much each probability would influence future events but this should be looked at and adjusted over time to monitor how the bias is shifting in the outcome distributions. Perhaps a GAN can be optimised to consider all of these factors in guessing of new numbers.

In [12]:
from random import randint
candidates = {}
for i in range(0, 1000):
    new_guess = []
    while len(new_guess) < 5:
        new_random = randint(1, 50)
        if new_random not in new_guess:
            new_guess.append(new_random)
    if (str(new_guess) in candidates):
        continue

    new_guess.sort()
    # print(new_guess)
    score = 0
    ave_score = 0
    rep_score = 0
    for j in range(len(new_guess)):
        if abs(running_probabilities[new_guess[j]] - ave) < (1.75*sd):
            ave_score += abs((abs(running_probabilities[new_guess[j]] - ave) - sd))
        else:
            ave_score += abs(running_probabilities[new_guess[j]] - ave)

        if (rep_averages[new_guess[j]] - rep_ave) < (1.45*rep_sd):
            rep_score += abs((abs(rep_averages[new_guess[j]] - rep_ave) - rep_sd))
        else:
            rep_score += abs(rep_averages[new_guess[j]] - rep_ave)
    # print(ave_score)
    # print(rep_score)

    guess_ave_dv = sum([
        new_guess[0],
        new_guess[1] - new_guess[0],
        new_guess[2] - new_guess[1],
        new_guess[3] - new_guess[2],
        new_guess[4] - new_guess[3],
        50 - new_guess[4]
    ]) / 6

    guess_ddv = sum([
        (guess_ave_dv - new_guess[0])**2,
        (guess_ave_dv - (new_guess[1] - new_guess[0]))**2,
        (guess_ave_dv - (new_guess[2] - new_guess[1]))**2,
        (guess_ave_dv - (new_guess[3] - new_guess[2]))**2,
        (guess_ave_dv - (new_guess[4] - new_guess[3]))**2,
        (guess_ave_dv - (50 - new_guess[4]))**2
    ])
    guess_sd_dv = math.sqrt(guess_ddv / 6)
    ddv_score = abs(sd_dv - guess_sd_dv) / (n / 5)
    # print(ddv_score)

    candidates[str(new_guess)] = ave_score + rep_score + ddv_score




In [13]:
candidates_ls = {}
for i in range(0, 1000):
    new_guess_ls = []
    while len(new_guess_ls) < 2:
        new_random = randint(1, 12)
        if new_random not in new_guess_ls:
            new_guess_ls.append(new_random)
    if (str(new_guess_ls) in candidates_ls):
        continue

    new_guess_ls.sort()
    # print(new_guess_ls)
    score = 0
    ave_score = 0
    rep_score = 0
    for j in range(len(new_guess_ls)):
        if abs(running_probabilities_ls[new_guess_ls[j]] - ave_ls) < (sd_ls):
            ave_score += abs((abs(running_probabilities_ls[new_guess_ls[j]] - ave_ls) - sd_ls))
        else:
            ave_score += abs(running_probabilities_ls[new_guess_ls[j]] - ave_ls)

        if (rep_averages_ls[new_guess_ls[j]] - rep_ave_ls) < (2*rep_sd_ls):
            rep_score += abs((abs(rep_averages_ls[new_guess_ls[j]] - rep_ave_ls) - rep_sd_ls))
        else:
            rep_score += abs(rep_averages_ls[new_guess_ls[j]] - rep_ave_ls)
    # print(ave_score)
    # print(rep_score)

    guess_ave_dv = sum([
        new_guess_ls[0],
        new_guess_ls[1] - new_guess_ls[0],
        12 - new_guess_ls[1]
    ]) / 3

    guess_ddv = sum([
        (guess_ave_dv - new_guess_ls[0])**2,
        (guess_ave_dv - (new_guess_ls[1] - new_guess_ls[0]))**2,
        (guess_ave_dv - (12 - new_guess_ls[1]))**2
    ])
    guess_sd_dv = math.sqrt(guess_ddv / 3)
    ddv_score = abs(sd_dv_ls - guess_sd_dv) / (n_ls / 2)
    # print(ddv_score)

    candidates_ls[str(new_guess_ls)] = ave_score + rep_score + ddv_score



200th to 240th best guesses for the ball numbers:

In [14]:
for i in sorted(candidates.items(), key=lambda item: item[1])[200:240]:
    print(i)

('[14, 20, 22, 28, 30]', 0.04858420057910213)
('[1, 12, 37, 39, 44]', 0.04863243012888356)
('[12, 24, 27, 41, 50]', 0.04888385023121399)
('[11, 24, 43, 45, 50]', 0.04890926511065989)
('[7, 9, 22, 29, 44]', 0.04895484395960152)
('[3, 17, 20, 46, 49]', 0.04905399354061182)
('[9, 14, 22, 29, 31]', 0.04910369380080541)
('[14, 19, 31, 33, 50]', 0.04911478728442667)
('[15, 19, 42, 46, 47]', 0.049146466022765616)
('[9, 10, 13, 14, 30]', 0.04915777962811295)
('[11, 14, 15, 18, 25]', 0.0491855325636643)
('[1, 6, 7, 16, 23]', 0.04926537988853577)
('[7, 25, 31, 47, 49]', 0.049277904649908395)
('[1, 20, 23, 24, 43]', 0.0492919539606823)
('[3, 11, 13, 28, 49]', 0.049331922983637165)
('[12, 16, 29, 39, 50]', 0.04940258822922929)
('[6, 22, 36, 40, 49]', 0.04944445518839116)
('[17, 22, 30, 41, 42]', 0.0495632109588092)
('[11, 16, 20, 22, 27]', 0.049702060125736865)
('[6, 8, 22, 29, 38]', 0.04971513666194079)
('[15, 25, 29, 41, 42]', 0.049777039937984366)
('[10, 14, 28, 37, 38]', 0.0498211382865372)
('

top guesses for the lucky star numbers:

In [15]:
for i in sorted(candidates_ls.items(), key=lambda item: item[1]):
    print(i)

('[9, 12]', 0.020060294401401443)
('[9, 10]', 0.021513470080736126)
('[10, 12]', 0.0240976066206148)
('[8, 9]', 0.03197541304472036)
('[8, 10]', 0.032022325400747564)
('[8, 12]', 0.03203425257314397)
('[4, 12]', 0.044235923489422714)
('[5, 12]', 0.04452021591210387)
('[4, 10]', 0.049685830327335714)
('[6, 12]', 0.05031425470491268)
('[7, 12]', 0.05205945704422598)
('[5, 10]', 0.053078430993794536)
('[4, 9]', 0.053667673240837065)
('[2, 10]', 0.05423836791712377)
('[7, 9]', 0.055619458039280534)
('[5, 9]', 0.05580291988728234)
('[2, 9]', 0.05595934186081061)
('[7, 10]', 0.056330124211570715)
('[1, 9]', 0.057359374971236596)
('[6, 10]', 0.05827886958285277)
('[6, 9]', 0.05882555743057614)
('[2, 12]', 0.060310682878184946)
('[1, 10]', 0.06160513192073648)
('[9, 11]', 0.06600410404237393)
('[1, 12]', 0.06719159659663075)
('[7, 8]', 0.06866484276130551)
('[5, 8]', 0.07051061980076645)
('[10, 11]', 0.07064358977276095)
('[1, 8]', 0.07217264599297561)
('[6, 8]', 0.07227590332404671)
('[11, 12

# GAN timeseries generation

In this section we will extract the data to be loaded into a neural net to make predictions and compare them to the existing ones we evaluated here. Eventually the idea is to use a GAN to enhance this prediction.